In [ ]:
import numpy as np
from numpy import asarray
from numpy import save
from numpy import load

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, Activation, Conv2D, BatchNormalization, ZeroPadding2D, MaxPool2D, GlobalAvgPool2D, Add, ReLU, Dense, AveragePooling2D, Flatten
from tensorflow.keras.initializers import glorot_uniform
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import glob
import random

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# Loading data
X_train = load('../input/gnr-contest/X_train_augmented.npy')
Y_train = load('../input/gnr-contest/Y_train_augmented.npy')

In [ ]:
a = 351
print(X_train.shape,Y_train.shape)
print(X_train[a].shape)

In [ ]:
Y_train2 = np.zeros(Y_train.shape[0])
for x in range(Y_train.shape[0]):
    Y_train2[x] = np.argmax(Y_train[x])+1
print(Y_train2.shape)

In [ ]:
a =310
plt.imshow(X_train[a]/255)
print(Y_train[a])
print(Y_train2[a])

In [ ]:
#Conv-BatchNorm-ReLU block

def conv_layer(x, filters, kernel_size, strides=1):
    
    x = Conv2D(filters=filters, kernel_size=kernel_size, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

In [ ]:
#Identity block

def res_identity(tensor, filters):
    
    x = conv_layer(tensor, filters=filters, kernel_size=1, strides=1)
    x = conv_layer(x, filters=filters, kernel_size=3, strides=1)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    
    x = Add()([tensor,x])    #skip connection
    x = ReLU()(x)
    
    return x

In [ ]:
#Projection block

def projection_block(tensor, filters, strides):
    
    #left stream
    x = conv_layer(tensor, filters=filters, kernel_size=1, strides=strides)
    x = conv_layer(x, filters=filters, kernel_size=3, strides=1)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    
    #right stream
    shortcut = Conv2D(filters=4*filters, kernel_size=1, strides=strides)(tensor)
    shortcut = BatchNormalization()(shortcut)
    
    x = Add()([shortcut,x])    #skip connection
    x = ReLU()(x)
    
    return x

In [ ]:
#Resnet block

def resnet_block(x, filters, reps, strides):
    
    x = projection_block(x, filters, strides)
    for _ in range(reps-1):
        x = res_identity(x,filters)
        
    return x

In [ ]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. We'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    
    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [ ]:
def ResNet50(input_shape = (64, 64, 3), classes = 2):   
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPool2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL.
    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
no_classes = 10
input_shape = (256,256,3)
model = ResNet50(input_shape = input_shape, classes = no_classes)

In [ ]:
def get_model(model_name):
  if model_name == "ResNet":
    inputs = keras.Input(shape=(256, 256, 3), name="img")
    x = conv_layer(inputs, filters=64, kernel_size=7, strides=2)
    x = MaxPool2D(pool_size = 3, strides =2)(x)
    x = resnet_block(x, filters=32, reps =3, strides=1)
    x = resnet_block(x, filters=64, reps =2, strides=2)
    x = resnet_block(x, filters=128, reps =3, strides=2)
    x = resnet_block(x, filters=256, reps =3, strides=2)
    x = GlobalAvgPool2D()(x)
    x = Dense(256,activation='relu')(x)
    outputs = Dense(10, activation ='softmax')(x)

    model = keras.Model(inputs, outputs, name="Resnet")
    model.summary()
    return model
    
  elif model_name == "ResNet-18":
    inputs = keras.Input(shape=(256, 256, 3), name="img")
    x = conv_layer(inputs, filters=64, kernel_size=7, strides=2)
    x = MaxPool2D(pool_size = 3, strides =2)(x)
    x = resnet_block(x, filters=64, reps =2, strides=1)
    x = resnet_block(x, filters=128, reps =2, strides=2)
    x = resnet_block(x, filters=256, reps =2, strides=2)
    x = resnet_block(x, filters=512, reps =2, strides=2)
    x = GlobalAvgPool2D()(x)
    x = Dense(100,activation='relu')(x)
    outputs = Dense(10, activation ='softmax')(x)

    model = keras.Model(inputs, outputs, name="Resnet")
    model.summary()
    return model
    
  elif model_name == 'VGG1':
    model = tf.keras.Sequential()

    # 1st Convolutional Layer
    model.add(layers.Conv2D(filters=64, kernel_size=(3,3),padding="Same",activation="relu" , input_shape = (256,256,3)))
    model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    # 2nd Convolutional Layer
    model.add(layers.Conv2D(filters=128, kernel_size=(3,3),padding="Same",activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    # 3rd Convolutional Layer
    model.add(layers.Conv2D(filters=256, kernel_size=(3,3),padding="Same",activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    # 4th Convolutional Layer
    model.add(layers.Conv2D(filters=256,kernel_size = (3,3),padding="Same",activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    # 5th Convolutional Layer
    model.add(layers.Conv2D(filters=512,kernel_size = (3,3),padding="Same",activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))

    model.add(layers.Flatten())

    # 1st Fully Connected Layer
    model.add(layers.Dense(512,activation="relu"))
    model.add(layers.Dropout(0.1))
    model.add(layers.BatchNormalization())

    # 2nd Fully Connected Layer
    model.add(layers.Dense(512,activation="relu"))
    model.add(layers.Dropout(0.1)) 
    model.add(layers.BatchNormalization())
    # Add output layer
    model.add(layers.Dense(10,activation="softmax"))
    return model

  elif model_name == "VGG2":
    inputs = keras.Input(shape=(256, 256, 3), name="img")
    x = layers.Conv2D(32, 3, activation="relu")(inputs)
    x = layers.Conv2D(64, 3, activation="relu")(x)
    x = layers.MaxPooling2D(3)(x)

    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)

    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = layers.MaxPooling2D(3)(x)

    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)

    x = layers.Conv2D(128, 3, activation="relu")(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10,activation="softmax")(x)

    model = keras.Model(inputs, outputs, name="Mini_VGGNet")
    model.summary()
    return model
  """elif model_name == "Inception v2":
  elif model_name == "Inception v3": 
  elif model_name == "Inception v4":  
  elif model_name == "DenseNet":
  elif model_name == "MobileNet":"""
  


In [ ]:
model_name = "VGG1"
model = get_model(model_name)
fig_name = model_name +".png"
#keras.utils.plot_model(model, fig_name, show_shapes=True)

In [ ]:
model = tf.keras.models.load_model("..input/dataset_name/model_name.h5")

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer = keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"],
)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

red_lr= ReduceLROnPlateau(monitor='val_loss',patience=4,verbose=1,factor=0.1, mode='min', min_lr=10**(-12))
mcp_save = ModelCheckpoint('.model_weights.hdf5', save_best_only=True, monitor='val_accuracy', mode='max')

In [ ]:
history = model.fit(X_train,Y_train,epochs = 15,validation_split = 0.2,batch_size = 64,shuffle = True,callbacks=[red_lr, mcp_save])

In [ ]:
#plotting the learning curve
import pandas as pd
pd.DataFrame(history.history).plot()

In [ ]:
model.save("VGG1.h5")

In [ ]:
preds = model.evaluate(X_train, Y_train)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
a =56
print(Y_train[a])
print(preds[a])
print(np.argmax(preds[a])+1)

In [ ]:
# prediction
images = load('../input/test-data/X_test.npy')
print("Shape of images: ",images.shape)
print("Number of images in class: ",images.shape[0])
print(images.shape)

In [ ]:
predictions = model.predict(images)
final_class_preds = []
no_images = images.shape[0]
for i in range(no_images):
  final_class_preds.append(np.argmax(predictions[i])+1)
print(len(final_class_preds))

In [ ]:
print(final_class_preds)

In [ ]:
#creating csv file
import csv
import pandas

f = open('./200050103.csv', 'w')
row = 'ImageID','LabelID'

# create the csv writer
writer = csv.writer(f)
writer.writerow(row)
for i in range(101,201):
  row = i,final_class_preds[i-101]
  # write a row to the csv file
  writer.writerow(row)

# close the file
f.close()

In [ ]:
submission = pd.DataFrame()
a= []
b=[]

for i in range(101,201):
    a.append(i)
    b.append(final_class_preds[i-101])
    
    
submission['ImageID'] = a
submission['LabelID']= b
    
submission.to_csv('prediction_vgg1_20epoch.csv', index=False)